In [1]:
#system imports
import time
import sys
sys.path.append('/home/saulo/projects/cpl/src')

#cpl imports
from cpl import cplconfig
from cpl.promotion import promote_instances
from cpl.utils import db_conn

In [9]:
def main():
    
    cpl_conf = cplconfig.configCPL()
    
    num_iter = cpl_conf.num_iter        # number of iterations
    max_p = cpl_conf.max_promotions     # max promotions per iteration
    l = cpl_conf.limit                  # max number of positive candidates for promotion
    T = cpl_conf.T                      # multiplier of promotion threshold
    
    db_cpl = db_conn('db_cpl')          # db connection to ontology
    
    categories = db_cpl.ontology.find()
    
    for i in range(num_iter):  # for i iterations
        
        i_start_time = time.time()
        
        if (i == 0):  #if first iteration
            promote_seed_instances(categories)
        else:
            for category in categories:  # for all categories
                
                if i < len(category['promoted_patterns']):  #if there are positive patterns for this iteration
                    start = time.time()
                    promote_instances(category, i, max_p, l, T)
                    end = time.time()
                    print(i, category_name, len(promote_instance), end-start, sep=',')
                    
        i_end_time = time.time()
        
        print('Iteration {} took {}s'.format(i, i_end_time-i_start_time))

main()
    

Iteration 0 took 0.04070568084716797s
Iteration 1 took 6.198883056640625e-06s
Iteration 2 took 5.9604644775390625e-06s
Iteration 3 took 5.4836273193359375e-06s
Iteration 4 took 5.0067901611328125e-06s
Iteration 5 took 5.245208740234375e-06s
Iteration 6 took 5.245208740234375e-06s
Iteration 7 took 5.4836273193359375e-06s
Iteration 8 took 5.245208740234375e-06s
Iteration 9 took 5.245208740234375e-06s


In [8]:
def promote_seed_instances(categories):
    db_cpl = db_conn('db_cpl')
    
    #category['category_name']
    
    for category in categories:
        db_cpl.ontology.update_one({'category_name':'city'},
                                   {"$set":{'promoted_instances.0':category['seed_instances']}})

In [ ]:
pr = promote_instance('city',2,10,20,4)

pr

In [63]:
pipeline = [{"$match":{"ctx_pattern":{"$in":["live in _","is mayor of _"]}}},
            {"$group": {"_id": "$noun_phrase", "count": {"$sum": "$counter"}}}]

res = db.allpairs.aggregate(pipeline)

list(res)

[{'_id': 'berlin', 'count': 1.0},
 {'_id': 'pittsburgh', 'count': 3.0},
 {'_id': 'las vegas', 'count': 1.0},
 {'_id': 'sao paulo', 'count': 4.0},
 {'_id': 'london', 'count': 1.0}]

In [91]:
x = [[1],[2],[3,4],[[5],[6,7,8]],[9],[10,11],[12],13]

list(flatten(x))

#list(itertools.chain.from_iterable(x))
#denest(x,out)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [11]:
def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

In [ ]:
def promote_instances(category, iteration, max_promotions ,limit, T):
    
    last_promoted_patterns = category['promoted_patterns'][iteration]
    all_promoted_patterns = list(flatten(category['promoted_patterns']))
    all_promoted_instances = list(flatten(category['promoted_instances']))
    
    #get the promoted patterns of the mutex exception categories
    mutex_query = (db_cpl.ontology.
                   find({'category_name':{"$in":category['mutex_exceptions']}}))
    
    mutex_patterns = list(flatten([i['promoted_patterns'] for i in mutex_query]))

    #count the ocurrences of all instances promoted pattern (positive) for this iteration
    pipe_pos = [{"$match":{"$and":[{"ctx_pattern":{"$in":last_promoted_patterns}},       # extraction step
                                   {"noun_phrase":{"$nin":all_promoted_instances}}]}},   # filter already promoted
                {"$group": {"_id": "$noun_phrase", "count": {"$sum": "$counter"}}}, 
                {"$sort": {"count":-1}},                                                 # rank step
                {"$limit": limit}]  # limit is used for performance only
    
    #count the ocurrences of all instances promoted pattern (negative) for this iteration: filter step
    pipe_neg = [{"$match":{"$and":[{"ctx_pattern":{"$nin":all_promoted_patterns}},
                                   {"ctx_pattern":{"$nin":mutex_patterns}}]}},
                {"$group": {"_id": "$noun_phrase", "count": {"$sum": "$counter"}}},
                {"$sort": {"count":-1}}]
    
    if list(db_ap.allpairs.aggregate(pipe_pos)):  #if at least one positive pattern was found
        df_positive = pd.DataFrame(list(db_ap.allpairs.aggregate(pipe_pos))).set_index("_id")
        df_negative = pd.DataFrame(list(db_ap.allpairs.aggregate(pipe_neg))).set_index("_id")
        
        joined = (df_positive.join(df_negative, rsuffix='_neg')
                  .fillna(0)
                  .assign(mult_count_neg=lambda df: df['count'] >= df['count_neg']*T))
        
        promoted_instances=list(joined[joined['mult_count_neg']]
                                .head(max_promotions)  # promote step
                                .index.values)
    else:
        promoted_instances = []
    
    return promoted_instances